In [5]:
#импортируем биб-ки, нужные для построения модели и графиков
import numpy as np #для работы с массивами и прочими мат.вычислениями
import pandas as pd #для работы с данными - датафреймы
import math
import matplotlib.pyplot as plt #для работы с графиками

Т.к. требуется найти оптимальное распределение отборов винограда с каждого виноградника такое, чтобы:
- Суммарная производительность по всем виноградникам была бы максимальной на 01.12.2021 - то будем использовать макс производит-ть каждого виноградника каждый день, тогда и 1.12.21 произв-ти также будут максимальными у каждого виноградника.
- Совокупные суточные отборы полностью покрывали бы потребность - выполнение этого условие обеспечит равномерное распределение отгрузки винограда с каждого виноградника в соответствии с определенным процентом от неоходимого объема. Как определим отбор процента с каждого виноградника: у нас есть по условию задачи две функции максимальных производительностей для каждого виноградника. Вычислим сколько максимально винограда можно забрать из виноградника 1 и из виноградника 2 на первый день работы, зная, что в обоих виноградниках по 450 млн куб.м ягод. В вычислениях (ниже) видно, что из виноградника 1 можно забрать 13,2125 млн куб.м ягод, а из виноградника 2 - 20,35 млн куб.м ягод. В процентном соотношении - это из виноградника 1 отбирается максимально 39,37% от общего нужного объема потребления винограда в день, а из виноградника 2 - (100-39,37)=60,63% от общего нужного объема потребления винограда в день.
- Суточные отборы по каждому винограднику не превышали бы максимальную производительность - это условие будем проверять при каждой итерации отбора.


In [6]:
#1. Сначала определим процент отбора исходя из функций максимальной прозводительности и объема винограда в каждом винограднике (ВНГ1 и ВНГ2 - обозначения соответственно виноградников 1 и 2):
x1=450 #начальный объем газа в первом винограднике, млн.куб.м
x2=450 #начальный объем газа во втором винограднике, млн.куб.м
f1=5+0.0000001*x1**3+ 0.00004*x1**2 - 0.02*x1
f2=10-0.00006*x2**2+ 0.05*x2
V=f1+f2
p1=round(f1*100/V, 2)
p2=100-p1
print("Из первого виноградника в первый день можно забрать {f} млн куб.м ягод, это составляет {p}% от общего необходимого объема потребления в день".format(f = f1, p = p1))
print("Из второго виноградника в первый день можно забрать {f} млн куб.м ягод, это составляет {p}% от общего необходимого объема потребления в день".format(f = f2, p = p2))

Из первого виноградника в первый день можно забрать 13.212499999999999 млн куб.м ягод, это составляет 39.37% от общего необходимого объема потребления в день
Из второго виноградника в первый день можно забрать 20.35 млн куб.м ягод, это составляет 60.63% от общего необходимого объема потребления в день


In [7]:
data = pd.read_csv('data_for_task3.csv', sep = ',') #считываем исходный файл в набор данных data
data = data.dropna() #удаляем нулевые значения из считанного набора данных
data.head()
x = data['x']
y = data['y']

In [8]:
df = pd.DataFrame(data) #загоняем в датафрейм
df

,x,y
0,2021-11-01,15.46
1,2021-11-02,15.93
2,2021-11-03,16.41
3,2021-11-04,16.90
4,2021-11-05,17.00
5,2021-11-06,17.00
6,2021-11-07,17.00
7,2021-11-08,17.00
8,2021-11-09,17.00
9,2021-11-10,17.00


In [9]:
date_array = [] #в этом массиве будут храниться данные о датах отгрузки винограда
vng1_array = [] #в этом массиве будут храниться объемы отгрузки винограда с первого виноградника на каждый день, максимум 39.37% от общего объема потребления
vng2_array = [] #в этом массиве будут храниться объемы отгрузки винограда со второго виноградника на каждый день, максимум 60,63% от общего объема потребления
v_need_array = [] #в этом массиве будут храниться нужные объемы потребления

v_need_array = data['y']

i_p1 = 0 #сколько млн куб.м ягод винограда забираем с первого виноградника (априори)
i_p2 = 0 #сколько млн куб.м газа забираем со второго виноградника (априори)
rest_vng1 = 450 #остаток винограда в первом винограднике
rest_vng2 = 450 #остаток винограда во втором винограднике

for i in v_need_array:
    i_p1 = i*p1/100 #p1 высчитан в начале программы
    i_p2 = i*p2/100 #p2 высчитан в начале программы
    f1=5+0.0000001*rest_vng1**3+ 0.00004*rest_vng1**2 - 0.02*rest_vng1 # - это сколько максимально млн.куб.м винограда можем взять в этот день с первого виноградника
    f2=10-0.00006*rest_vng2**2+ 0.05*rest_vng2 # - это сколько максимально млн.куб.м винограда можем взять в этот день со второго виноградника
    if f1 >= i_p1 and f2 >= i_p2 and i_p1 <= rest_vng1 and i_p2 <= rest_vng2: #т.е. объем отбора с каждого виноградника не должен превышать максимальную произв-ть (это третье условие в задаче)
        vng1_array.append(i_p1) #записали объем отгрузки винограда с первого виноградника в этот день
        vng2_array.append(i_p2) #записали объем отгрузки винограда со второго виноградника в этот день
    else:     
        #иначе, если стандартный процент отбора (i_p1, i_p2) превышает макс.произв-ть виноградников (f1, f2), то применяем вычисленный максимально возможный объем отгрузки с каждого виноградника:
        if f1 < i_p1 and f2 >= i_p2: #когда максимум с первого виноградника меньше нужного процента отгрузки из него
            i_p1 = f1
            i_p2 = i - f1
            vng1_array.append(i_p1) #забираем допустимый максимум с первого виноградника
            vng2_array.append(i_p2) #остальное забираем допустимый максимум со второго виноградника
        elif f1 >= i_p1 and f2 < i_p2: #когда максимум со второго виноградника меньше нужного процента отгрузки из него
            i_p2 = f2
            i_p1 = i - f2
            vng1_array.append(i_p1) #забираем допустимый максимум с первого виноградника
            vng2_array.append(i_p2) #остальное забираем допустимый максимум со второго виноградника
    rest_vng1 = rest_vng1 - i_p1 #уменьшаем кол-во винограда в остатке в первом винораднике после отгрузки
    rest_vng2 = rest_vng2 - i_p2 #уменьшаем кол-во винограда в остатке во втором винораднике после отгрузки    
            
df_vng1 = pd.DataFrame(vng1_array) #переводим массив объемов отгрузок винограда с первого виноградника в датафрейм
df_vng2 = pd.DataFrame(vng2_array) #переводим массив объемов отгрузок винограда со второго виноградника в датафрейм

df_test_z = pd.DataFrame() #заводим новую оболочку под х и расчётный у (y - это искомая цена)
df_test_z['Дата'] = df.x #загоняем в df_test_z['x'] даты
df_test_z['Объемы потребления с первого виноградника'] = df_vng1
df_test_z['Объемы потребления со второго виноградника'] = df_vng2


In [10]:
df_test_z #просто выведем на экран, чтобы посмотреть 

,Дата,Объемы потребления с первого виноградника,Объемы потребления со второго виноградника
0,2021-11-01,6.086602,9.373398
1,2021-11-02,6.271641,9.658359
2,2021-11-03,6.460617,9.949383
3,2021-11-04,6.653530,10.246470
4,2021-11-05,6.692900,10.307100
5,2021-11-06,6.692900,10.307100
6,2021-11-07,6.692900,10.307100
7,2021-11-08,6.692900,10.307100
8,2021-11-09,6.692900,10.307100
9,2021-11-10,6.692900,10.307100


In [11]:
#пишем в файл результаты:
#в первом столбце из датафрейма df_test_z - хранится дата, во втором столбце - хранится прогнозное значение объема отгрузки для первого виноградника, в третьем столбце - хранится прогнозное значение объема отгрузки для второго виноградника
df_test_z.to_csv("grapes_volume_distribution_task3.csv", index=False, encoding='utf-8-sig')
print("Результаты распределения объемов потребления винограда выгружены в файл grapes_volume_distribution_task3.csv")
print("Файл хранится в той же папке, откуда запускается эта IDE")

Результаты распределения объемов потребления винограда выгружены в файл grapes_volume_distribution_task3.csv
Файл хранится в той же папке, откуда запускается эта IDE
